## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from collab_splats.wrapper import Splatter, SplatterConfig
import pyvista as pv

# pv.start_xvfb()


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [ ]:
mesher_kwargs = {
    'depth_name': "median_depth",
    'depth_trunc': 1.0, # Should be between 1.0 and 3.0
    'voxel_size': 0.01, 
    'normals_name': "normals",
    'features_name': "distill_features", 
    'sdf_trunc': 0.03,
    'clean_repair': True,
    'align_floor': True,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    overwrite=True
)


Available runs:
[0] 2025-07-11_171420
Initializing mesher Open3DTSDFFusion


[17:59:17] Auto image downscale factor of 2                                                 ]8;id=567008;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=924104;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[17:59:46] use color only optimization with sigmoid activation                                         ]8;id=220288;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=914046;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

...

Skipping hole Id_EdgeTag(27) of perimeter 45.35377921021609


Filling holes (822): 100%|██████████| 822/822 [00:08<00:00, 98.96it/s] 


Mapping normals to mesh


Finished computing mesh: 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/mesh/Open3dTSDFfusion.ply

Mapping features to mesh


### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [14]:
splatter.plot_mesh(attribute="RGB")

Number of points: 102294
Number of cells: 200193
Bounds: BoundsTuple(x_min=-0.5049999952316284, x_max=0.7450000047683716, y_min=-0.125, y_max=1.4850000143051147, z_min=-1.0152238607406616, z_max=0.9950358867645264)


Widget(value='<iframe src="http://localhost:41857/index.html?ui=P_0x7ba9aff6bb50_6&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [15]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Plot similarity maps

In [16]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 102294
Number of cells: 200193
Bounds: BoundsTuple(x_min=-0.5049999952316284, x_max=0.7450000047683716, y_min=-0.125, y_max=1.4850000143051147, z_min=-1.0152238607406616, z_max=0.9950358867645264)


Widget(value='<iframe src="http://localhost:41857/index.html?ui=P_0x7ba9aff6ba00_7&reconnect=auto" class="pyvi…